In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.7 MB/s eta 0:00:00


In [ ]:
!pip install langchain_openai

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

@tool
def write_file(file_name: str, content: str) -> None:
    """Escreve o conteúdo em um arquivo"""
    with open(file_name, "w") as f:
        f.write(content)

tools = [weather, multiply, write_file]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente mas não sabe do clima ou fazer cálculos matemáticos.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "crie um arquivo com informações sobre o clima em Recife?"})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Recife'}`


ensolarado
Invoking: `write_file` with `{'file_name': 'clima_recife.txt', 'content': 'Informações sobre o clima em Recife:\n\n- Condição: Ensolarado'}`


NoneO arquivo com informações sobre o clima em Recife foi criado com sucesso. Ele contém a seguinte informação:

```
Informações sobre o clima em Recife:

- Condição: Ensolarado
```

> Finished chain.


{'input': 'crie um arquivo com informações sobre o clima em Recife?',
 'output': 'O arquivo com informações sobre o clima em Recife foi criado com sucesso. Ele contém a seguinte informação:\n\n```\nInformações sobre o clima em Recife:\n\n- Condição: Ensolarado\n```'}

## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

In [ ]:
import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

@tool
def write_file(file_name: str, content: str) -> None:
    """Escreve o conteúdo em um arquivo"""
    with open(file_name, "w") as f:
        f.write(content)

tools = [weather, multiply, write_file]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "crie um arquivo com informações sobre o clima em Recife?"})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Recife'}`


nublado
Invoking: `write_file` with `{'file_name': 'clima_recife.txt', 'content': 'Informações sobre o clima em Recife:\n\n- Condição: Nublado'}`


NoneO arquivo com informações sobre o clima em Recife foi criado com sucesso. Ele contém a seguinte informação:

- Condição: Nublado

> Finished chain.


{'input': 'crie um arquivo com informações sobre o clima em Recife?',
 'output': 'O arquivo com informações sobre o clima em Recife foi criado com sucesso. Ele contém a seguinte informação:\n\n- Condição: Nublado'}

### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

In [ ]:
import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

@tool
def write_file(file_name: str, content: str) -> None:
    """Escreve o conteúdo em um arquivo"""
    with open(file_name, "w") as f:
        f.write(content)

@tool
def count_letter(word: str, letter: str) -> int:
    """Conta a quantidade de ocorrências de uma letra em uma palavra"""
    count = 0
    for l in word:
        if l == letter:
            count += 1
    return count

tools = [weather, multiply, write_file, count_letter]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Conte quantas vezes tem a letra 'e' em Recife"})



> Entering new AgentExecutor chain...

Invoking: `count_letter` with `{'word': 'Recife', 'letter': 'e'}`


2A letra 'e' aparece 2 vezes na palavra "Recife".

> Finished chain.


{'input': "Conte quantas vezes tem a letra 'e' em Recife",
 'output': 'A letra \'e\' aparece 2 vezes na palavra "Recife".'}